# Keys in Real Data: Cabrillo Courses 

In this lab we will examine some trade-offs when addressing problems with data. This lab builds on the schema that we imported in the [Using Real Data: Cabrillo Courses](http://www.lifealgorithmic.com/content/intro-to-rdbms/labs/realdata_1_cabrillo_import.html) lab. This is the first time I've used Jupyter for this lab and I've discovered that it's considerably faster. 

As usual, start by running setup code in the next cell: 

In [ ]:
%load_ext sql
%config SqlMagic.autolimit=500
    
import pandas as pd 
from sqlalchemy import create_engine

## Step 0: Import the Data (The Jupyter Way)

Upload the CSV files that I provided into Jupyter Lab by dragging and dropping them from your computer to the files panel. The code in the next cell imports the CSVs and creates tables with corresponding names. 


In [ ]:
datafiles = ['MasterCourseFile.csv', 'ProgramFile.csv', 'ProgramCourseFile.csv']
url = 'sqlite:///real_data_2.sqlite3'
engine = create_engine(url, echo=False)
for f in datafiles:
    df = pd.read_csv(f)
    df.to_sql(f.replace('.csv',''), con=engine, if_exists='replace')

**Much faster!!** Now verify that your tables are loaded using the query in the next cell:

In [ ]:
%%sql sqlite:///real_data_2.sqlite3

select * from ProgramFile

## Cleanup (if necessary) 

This cell enables you to re-run the whole notebook by removing the created tables:

In [ ]:
%%sql sqlite:///real_data_2.sqlite3

drop table if exists program_course; 
drop table if exists program; 
drop table if exists course;

## Step 1: Course Data 

The course data is luckily clean. Let's recreate the course table with a key: 

In [ ]:
%%sql sqlite:///real_data_2.sqlite3

PRAGMA foreign_keys = ON;

drop table if exists course; 
create table course (
    ControlNumber char(12) primary key,
    CourseID char(12),
    TOPCode char(6), 
    CreditStatus char(1), 
    MaximumUnits float,
    MinimumUnits float, 
    SAMCode char(1), 
    Date date
);

insert into course (ControlNumber, CourseID, TOPCode,
        CreditStatus, MaximumUnits, MinimumUnits, 
        SAMCode, Date)
    select `Control Number`, `Course ID`, `TOP Code`, 
        `Credit Status`, `Maximum Units`, `Minimum Units`, 
        `SAM Status`, `Issue/Update Date`
    from MasterCourseFile;

## Step 2: Keys in the Program Data 

This query shows the problem with the program data: 

In [ ]:
%%sql sqlite:///real_data_2.sqlite3

select `Program Control Number`, count(*)  
    from ProgramFile
    group by `Program Control Number`
    having count(*) > 1;

In [ ]:
%%sql sqlite:///real_data_2.sqlite3

select * from ProgramFile
    where `Program Control Number` like '     ';

Some records don't have a control number. What can we do about it? Let's delete the rows and re-create the program table with a key.

In [ ]:
%%sql sqlite:///real_data_2.sqlite3

drop table if exists program;
create table program ( 
    ControlNumber char(5) primary key,
    Title varchar(64),
    TOPCode char(6), 
    AwardType char(1), 
    CreditType char(1), 
    ApprovedDate date, 
    Status varchar(16), 
    InactiveDate date
);

insert into program (ControlNumber, Title, TOPCode, AwardType, CreditType, ApprovedDate, Status, InactiveDate)
    select `Program Control Number`, `Title`, `TOP Code`, `Program Award`, 
        `Credit Type`, IIF (`Approved Date` = '', NULL, `Approved Date`), 
        TRIM(`Proposal Status`), IIF (`Inactive Date` = '', NULL, `Inactive Date`)
    from ProgramFile
    where `Program Control Number` != '     ';


Note the `where` clause in the insert/select statement removes rows with empty control numbers.

## Step 3: Keys in the Intersection Table 

Now let's fix the `program_course` table to use our new key. This too will encounter problems with empty control numbers. Run this query to check for bad keys: 

In [ ]:
%%sql sqlite:///real_data_2.sqlite3

select `Program Control Number`, `Course Control Number`, count(*) 
    from ProgramCourseFile 
    group by `Program Control Number`, `Course Control Number`
    having count(*) > 1;

There are courses that map to empty programs. Again, we're going to discard this data because it's hard to figure out where these courses belong.

In [ ]:
%%sql sqlite:///real_data_2.sqlite3

drop table if exists program_course;
create table program_course (
    ProgramControlNumber char(5), 
    CourseControlNumber char(12),
    primary key (ProgramControlNumber, CourseControlNumber),
    foreign key (ProgramControlNumber) 
        references program (ControlNumber),
    foreign key (CourseControlNumber) 
        references course (ControlNumber)
);

insert into program_course (ProgramControlNumber, CourseControlNumber)
    select `Program Control Number`, `Course Control Number` 
    from ProgramCourseFile
    where `Program Control Number` != '     ';    

## Turn In 

Download the `real_data_2.sqlite3` file and submit it on Canvas.